In [ ]:
#utils for algorithms

import numpy as np

def euclidianDistance(point, candidate):
    return np.linalg.norm(point - candidate)

def distances(point, points):
    return [euclidianDistance(point, candidate) for candidate in points]

def chooseNewCentroid(centroids, cluster, i):
    if len(cluster) == 0 :
        return centroids[i]
    else:
        return np.mean(cluster,axis=0)


In [ ]:
def kMeansShell(dataset, metric, cluster_count, starting_strategy, iterations):
    def iteration(centroids):
        distance = np.linalg.norm(dataset[:, np.newaxis] - centroids, axis=2)
        labels = np.argmin(distance, axis=1)
            
        indecies = [[i for i,x in enumerate(labels) if x == j ] for j in range(cluster_count)]
        clusters = [[p for p in dataset[indecies[i]] ] for i in range(cluster_count)]
        new_centroids = [chooseNewCentroid(centroids, cluster, i) for i, cluster in enumerate(clusters)]
        change = np.sum([min(distances(p, centroids)) for p in new_centroids])

        return new_centroids, labels, clusters, change

    best_thus_far = np.inf
    for i in range(iterations):
        current_centroids = starting_strategy(dataset, cluster_count)
        change = 1
        while change != 0:
            current_centroids, labels, clusters, change = iteration(current_centroids)
        
        new_metric_result = metric(clusters, current_centroids)
        best_thus_far = min(best_thus_far, new_metric_result)
        if best_thus_far == new_metric_result:
            result_centroids = current_centroids
            result_labels = labels
            # np.savetxt('centroids.txt', np.array(result_centroids))
            # np.savetxt('cluster_labels.txt', np.array(result_labels), fmt="%d")
            # %run plot_clusters.py Datasets/unbalance/unbalance.txt centroids.txt cluster_labels.txt

    np.savetxt('centroids.txt', np.array(result_centroids))
    np.savetxt('cluster_labels.txt', np.array(result_labels), fmt="%d")


In [499]:
def kMeans(dataset, metric, cluster_count):
    def choosing(dataset, cluster_count):
        return dataset[np.random.permutation(len(dataset))[:cluster_count]]
    
    kMeansShell(dataset, metric, cluster_count, choosing,20)

def kMeansplusplus(dataset, metric, cluster_count):
    def choosing(dataset, cluster_count):
        result = [dataset[np.random.choice(range(len(dataset)))]]
        while len(result) < cluster_count:
            candidateDistances = [np.min(distances(current, result)) for current in dataset]
            result.append(dataset[np.argmax(candidateDistances)])

        return result
    
    kMeansShell(dataset, metric, cluster_count, choosing,1)



In [500]:
def wcss(clusters, centroids):
    return np.sum([np.sum(distances(centroids[i], clusters[i])) for i, _ in enumerate(centroids)])

def daviesBouldinIndex(clusters, centroids):
    size = len(centroids)
    s = [np.sum(distances(centroids[i], cluster))*1.0/len(cluster) for i,cluster in enumerate(clusters)]
    m = [[euclidianDistance(centroids[i], centroids[j]) for j in range(size)] for i in range(size)]

    return np.sum([np.max([(s[i]+s[j])/m[i][j] for j in range(size) if j != i]) for i in range(size)])*1.0/size


In [501]:
# utils for main

def chooseAlgorithm(name):
    pairs = {
        "kMeans" : kMeans,
        "kMeans++" : kMeansplusplus,
    }

    return pairs[name]

def chooseMetric(number):
    pairs = {
        "1" : wcss,
        "2" : daviesBouldinIndex,
    }

    return pairs[number]


In [502]:
import sys
#inp = input("<data_file> <algorithm_name> <metric> <cluster_count>")
inp = "unbalance.txt kMeans 2 8"
labels = inp.split(" ")
if len(labels) != 4:
    sys.exit("Not a valid input")

algorithm = chooseAlgorithm(labels[1])
metric = chooseMetric(labels[2])

labels[0] = "Datasets\\" + labels[0][:-4] + "\\" + labels[0]

algorithm(np.loadtxt(labels[0]) ,metric, int(labels[3]))

%run plot_clusters.py {labels[0]} centroids.txt cluster_labels.txt


[[  4231.8083605   41811.92692283   3375.01481478 ...  53917.39735929
    3495.73068185  56493.13021952]
 [  8817.96195274  36668.51687756   8483.34945644 ...  49911.21642477
    8407.55017826  52394.5886996 ]
 [  4799.22920895  47095.71320195   8390.60486497 ...  62775.76960102
    5734.54828212  65312.01048046]
 ...
 [401618.00768641 367553.64278565 399532.7102416  ... 343894.0788237
  400955.61577935 341181.86125584]
 [396694.20771168 362471.34619029 394629.21331929 ... 339015.51377481
  396036.48645674 336295.52292589]
 [382677.3407768  348258.1543008  380640.86450222 ... 325067.04832696
  382026.13590303 322335.9827323 ]]
[[  4775.20377172  40421.51686104   3083.83661622 ...  53768.415433
    1798.59549345 145319.57471732]
 [  8671.92481642  35233.96594781   7749.82275723 ...  49814.38812895
    6523.74918021 141266.21440281]
 [  4403.48228199  46055.529243     9655.09925499 ...  62646.78497078
    7191.7492154  154174.67655131]
 ...
 [401688.32677062 367095.87241455 398198.692224

C:\Users\bugyb\AppData\Local\Temp\ipykernel_9948\2857484118.py:6: RuntimeWarning: invalid value encountered in scalar divide
  s = [np.sum(distances(centroids[i], cluster))*1.0/len(cluster) for i,cluster in enumerate(clusters)]


[[ 60341.10131659  37255.6551457   56212.36531729 ...  42859.91338292
  338340.13990835  41951.53074183]
 [ 56322.31800993  32081.19065757  52376.27721967 ...  37717.57024624
  334270.06992053  36725.35344516]
 [ 69197.21872587  42833.85500583  65130.17243066 ...  48120.82626997
  347193.04676014  48023.51211558]
 ...
 [337518.43258739 369800.76092392 342030.75781697 ... 366875.4267639
   72402.85878773 363999.6506561 ]
 [332645.60952894 364740.8606547  337173.54982763 ... 361788.90756691
   69605.44197696 358936.98183022]
 [318706.13921654 350552.5056092  323256.74648547 ... 347571.17781231
   60933.70970357 344745.69037152]]
[[ 60346.59216841  37250.5723181   56154.46990363 ...  42848.13632151
  338340.13990835  41956.66585421]
 [ 56331.30109633  32075.86007487  52316.78699041 ...  37706.31920187
  334270.06992053  36730.58349342]
 [ 69204.15427725  42830.76272365  65071.79689066 ...  48105.77417076
  347193.04676014  48026.7678641 ]
 ...
 [337520.60628937 369796.8719578  342083.0270

KeyboardInterrupt: 